In [1]:
import math
import numpy as np

# Elliptical orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  In this notebook I have begun to implement elliptical orbits. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

In [2]:
def getfocus(a,ecc):
    c=a*ecc
    return c

In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
radius=10
t=2*math.pi*0.01*np.arange(103)
a=10
ecc=.5
b=a*np.sqrt(1-ecc**2)
centerx=0
centery=0
focusx=centerx+getfocus(a,ecc)
focusy=centery+0.0
p = figure(title="Ellipse", plot_width=400, plot_height=400, x_range=[-15,15], y_range=[-15,15])
p.line(centerx+a*np.cos(t),centery+b*np.sin(t),line_width=3,line_color="blue",legend="Circle")
p.circle(focusx,focusy,line_width=15,line_color="green",legend="Focus 1")
p.circle(centerx-getfocus(a,ecc),focusy, line_width=15, line_color="red", legend="Focus 2")
p.legend.location="bottom_right"
show(p)

Loading BokehJS ...

In [4]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [5]:
import math
def InitialDataEqualMassConic(radius,ecc,angle,initmass):
    orbitalangle=angle
    print("angle",angle)
    phi=np.array([math.pi+orbitalangle,orbitalangle])
    orbitalradius=radius #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [6]:
import random,numpy as np
rad0=50
ecc0=0.5
theta0=0. #math.pi/6.
mass0=1.0
initdateqellipse=InitialDataEqualMassConic(rad0,ecc0,theta0,mass0)
print(initdateqellipse)

angle 0.0
0.5
(array([3.14159265, 0.        ]), 0.0, 50, 0.5, array([1., 1.]))


In [7]:
def getxyuveqtwoellipses(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, eccentricity, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ux0=0.
    uy0=0.
    uz0=0.
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    #masses=np.zeros(2)
    #masses[0]=masses[1]+masses[0]
    #masses[1]=(masses[1]*masses[0])/(masses[1]+masses[0])
    if eccentricity<1 and eccentricity>0:
        #elliptical
        orbitalr=orbitalradius #NO REDUCED MASS 
        #focusdisp=getfocus(orbitalr,eccentricity) #displacement of the center due to the focus being at the center
        coordsep=orbitalr*(1+eccentricity) #start at aphelion
        x0=(coordsep)*cosphi
        y0=(coordsep)*sinphi
        starsep=np.sqrt((x0[0]-x0[1])**2+(y0[0]-y0[1])**2) #two stars, at opposite ends of the orbit
        Fapastron=masses[1]*masses[0]/starsep**2
        #print('xycomp',coordsep,focusdisp,x0,y0)
        #x0[0]=0.0
        #y0[0]=0.0
        v=np.zeros(2)
        #v= np.sqrt(masses[1]*masses[0]/masses*(2./coordsep-1./(orbitalr)))
        v= np.sqrt(masses[1]*masses[0]/masses*(2./starsep-1./(2.*orbitalr)))
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        print("vstuff", coordsep,orbitalr, 2./coordsep, 1./orbitalr, (2./starsep-1./(2.*orbitalr)), 1./(coordsep))
        
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v[0]*sinphi
        uy0=v[1]*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a[0]*cosphi
        ay0=-a[1]*sinphi
        az0=np.zeros(2)

    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [8]:
xyuvaeqtwoellipses=getxyuveqtwoellipses(initdateqellipse)
print(xyuvaeqtwoellipses)

initdat 50 [3.14159265 0.        ] 0.5 [-1.  1.] [1.2246468e-16 0.0000000e+00]
coordsep 50
pi
zero
[-50.  50.]
[0. 0.]
0.5
vstuff 75.0 50 0.02666666666666667 0.02 0.003333333333333334 0.013333333333333334
(array([1., 1.]), array([-75.,  75.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([-0.05773503,  0.05773503]), array([0., 0.]), array([ 4.44444444e-05, -4.44444444e-05]), array([-0., -0.]), array([0., 0.]))


(array([1., 1.]), array([ 50., -50.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 0.07071068, -0.07071068]), array([0., 0.]), array([-2.5e-05,  2.5e-05]), array([-0., -0.]), array([0., 0.]))
Circular

In [9]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    

    Gconstant=1
    for star1 in np.arange(len(mass)):
        for star2 in np.arange(len(mass)):
            if star1!=star2:
                #Depends on the separation vector's magnitude, rsepjk, because gravity is a central force, 
                #not because it moves in a circle. 
                #This should be true even if it moves in an ellipse. This is Newton's Force Law.
                #Newton's force law applies in the non-relativisitic limit. 
                # In this limit, the orbits are stable and no inspiral or outspiral occurs and 
                # no gravitational waves are emitted.
                #In vector form, broken into its components
                
                
                #BECAUSE ACCELERATION IS DECREASING WHILE RADIUS IS INCREASING FOR AN APHELION START, THE PROBLEM 
                #HAS TO BE BETWEEN HERE AND 
                xvec=xi[star2]-xi[star1]
                yvec=yi[star2]-yi[star1]
                zvec=zi[star2]-zi[star1]
                rvecsq=xvec**2+yvec**2+zvec**2
                rvec3halves=rvecsq**(1.5)
                if step < 10:
                    print(xvec,yvec,zvec,rvecsq,rvec3halves)
                #xhat=xvec/rvec #projection in the x direction
                #yhat=yvec/rvec #projection in the y direction
                #zhat=zvec/rvec #projection in the z direction
                #if step< 10:
                #    print(xhat,yhat,zhat)
                NewtonF=mass[star1]/rvec3halves
                axii[star2]=-NewtonF*xvec
                ayii[star2]=-NewtonF*yvec
                azii[star2]=-NewtonF*zvec
                #HERE
                #print(yi[star2],yi[star1],yvec,rvec,yhat,NewtonF,ayii[star2])
    if step<10:
        print("ax",axii)
        print("ay",ayii)
        print("az",azii)
        print("a",axii**2+ayii**2)
    for m in np.arange(len(mass)):
        #m represents choices of mass
        i=step
        dx_by_dt=vxi[m]
        dy_by_dt=vyi[m]
        dz_by_dt=vzi[m]
        dx_by_dt_initial=dx_by_dt
        dy_by_dt_initial=dy_by_dt
        dz_by_dt_initial=dz_by_dt
        d2x_by_dt2=axi[m]
        d2y_by_dt2=ayi[m]
        d2z_by_dt2=azi[m]
        yinitial=yi[m]
        xinitial=xi[m]
        zinitial=zi[m]
        
        xfinal = xinitial + dt*dx_by_dt
        dx_by_dt_final = dx_by_dt_initial + dt*d2x_by_dt2
        yfinal= yinitial + dt*dy_by_dt
        dy_by_dt_final = dy_by_dt_initial + dt*d2y_by_dt2
        zfinal= zinitial + dt*dz_by_dt
        dz_by_dt_final = dz_by_dt_initial + dt*d2z_by_dt2
        #rfinal=np.sqrt(xfinal**2+yfinal**2+zfinal**2)
        
        xii[m]=xfinal
        yii[m]=yfinal
        zii[m]=zfinal
        vxii[m]=dx_by_dt_final
        vyii[m]=dy_by_dt_final
        vzii[m]=dz_by_dt_final
        #rii[m]=rfinal
   


    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [10]:
def getPotentialEnergy(x1,y1,x2,y2, mass1,mass2):
    diffx=x1-x2
    diffy=y1-y2
    r=np.sqrt(diffx**2+diffy**2)
    return -mass1*mass2/r**2

In [11]:
def getKineticEnergy(vx,vy, mass):
    return 1/2*mass*(vx**2+vy**2)

In [12]:
def getAngularMomentum(x,y,vx,vy,mass):
    return mass*(vx*y-vy*x)

In [13]:
dt=1 #*31556926 #seconds per year
numsteps=100000
savestep=100
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqtwoellipses
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
vxarr1=[]
vxarr2=[]
vyarr1=[]
vyarr2=[]
axarr1=[]
ayarr1=[]
axarr2=[]
ayarr2=[]
Earr=[]
Larr=[]
t=0.0
tarr=[]
masstotal=1.
L=np.zeros(len(mass0))
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    PE=getPotentialEnergy(x[0],y[0],x[1],y[1], mass[0],mass[1])
    KE=0.
    for j in np.arange(len(mass)):
        KE+=getKineticEnergy(vx[j],vy[j], mass[j])
    L[0]=getAngularMomentum(x[0],y[0],vx[0],vy[0],mass[0])
    E=PE+KE
    #print(x,y,vx,vy,ax,ay,E,L)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    t+=dt
    if i%savestep==0:
        xcoord1.append(x[0])
        xcoord2.append(x[1])
        ycoord1.append(y[0])
        ycoord2.append(y[1])
        vxarr1.append(vx[0])
        vxarr2.append(vx[1])
        vyarr1.append(vy[0])
        vyarr2.append(vy[1])
        axarr1.append(vx[0])
        axarr2.append(vx[1])
        ayarr1.append(vy[0])
        ayarr2.append(vy[0])
        Earr.append(E)
        Larr.append(L[0])
        tarr.append(t)
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[-75.  75.]
150.0 0.0 0.0 22500.0 3375000.0
-150.0 0.0 0.0 22500.0 3375000.0
ax [ 4.44444444e-05 -4.44444444e-05]
ay [-0. -0.]
az [-0. -0.]
a [1.97530864e-09 1.97530864e-09]
150.0 0.11547005383792516 0.0 22500.013333333332 3375003.0000004442
-150.0 -0.11547005383792516 0.0 22500.013333333332 3375003.0000004442
ax [ 4.44444049e-05 -4.44444049e-05]
ay [ 3.42133189e-08 -3.42133189e-08]
az [-0. -0.]
a [1.9753063e-09 1.9753063e-09]
149.99991111111112 0.23094010767585033 0.0 22500.02666667457 3375006.0000035563
-149.99991111111112 -0.23094010767585033 0.0 22500.02666667457 3375006.0000035563
ax [ 4.44443391e-05 -4.44443391e-05]
ay [ 6.84265769e-08 -6.84265769e-08]
az [-0. -0.]
a [1.97530396e-09 1.97530396e-09]
149.99973333333335 0.34641016151377546 0.0 22500.040000071116 3375009.000020001
-149.99973333333335 -0.34641016151377546 0.0 22500.040000071116 3375009.000020001
ax [ 4.44442469e-05 -4.44442469e-05]
ay [ 1.02639774e-07 -1.02639774e-07]
az [-0. -0.]
a [1.97530162e-09 1.97530162e-09]
149

print(yi[star2],yi[star1],yvec,rvec,yhat,NewtonF,ayii[star2])

In [14]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, e=0.5", plot_width=400, plot_height=400, y_range=[-350,350],x_range=[-350,350])
t = np.linspace(0, 2*math.pi, 100)
focus=getfocus(rad0,ecc0)
b=rad0*np.sqrt(1-ecc**2)
print(b)
print(focusx)
p.line(xcoord1,ycoord1, line_color="purple",legend="Ellipse 1")
p.line(xcoord2,ycoord2, line_color="blue",legend="Ellipse 2")

#p.line(focusx+rad0*np.cos(t),b*np.sin(t), line_color="green", legend="Ideal ellipse 1")
#p.line(-focusx+rad0*np.cos(t),b*np.sin(t), line_color="orange", legend="Ideal ellipse 2")
p.line(focus*np.cos(theta0)+rad0*np.cos(t)*np.cos(theta0)-b*np.sin(t)*np.sin(theta0),focus*np.sin(theta0)+rad0*np.cos(theta0)*np.sin(t)+b*np.sin(theta0)*np.cos(t), line_color="cyan", legend="Ideal ellipse 1")
p.line(-focus*np.cos(theta0)+rad0*np.cos(t)*np.cos(theta0)-b*np.sin(t)*np.sin(theta0),-focus*np.sin(theta0)+rad0*np.cos(theta0)*np.sin(t)+b*np.sin(theta0)*np.cos(t), line_color="black", legend="Ideal ellipse 2")

print(theta0)
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

43.30127018922193
5.0
0.0


In [15]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
print(len(Larr))
print(len(tarr))
print(len(Earr))
print(max(Larr))
print(max(Earr))

# create a new plot with default tools, using figure
p = figure(title="Energy", plot_width=400, plot_height=400)
p.line(tarr,np.array(Earr), line_color="purple",legend="Energy")
p.line(tarr,np.array(Larr)-max(Larr),line_color="blue", legend="Angular Momentum")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

999
999
999
-4.330637660316054
0.0032938185187060425


In [16]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
print(len(Larr))
print(len(tarr))
print(len(Earr))
print(max(Larr))
print(max(Earr))

# create a new plot with default tools, using figure
p = figure(title="Angular Momentum", plot_width=400, plot_height=400)
#p.line(tarr,1000*np.array(vxarr1), line_color="purple",legend="External")
#p.line(tarr,np.array(ycoord1),line_color="green")
#p.line(tarr,1000*np.array(vyarr1), line_color="blue")
#p.line(tarr,np.array(xcoord1), line_color="yellow")
p.line(tarr,np.array(vxarr1)*np.array(ycoord1), line_color="orange", legend="vx*y")
p.line(tarr,np.array(vyarr1)*np.array(xcoord1),line_color="black", legend="vy*x")
p.line(tarr,(np.array(vxarr1)*np.array(ycoord1)-np.array(vyarr1)*np.array(xcoord1)), line_color="cyan", legend="L external")
#p.line(tarr,np.array(Larr),line_color="blue", legend="Internal")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

999
999
999
-4.330637660316054
0.0032938185187060425


In [17]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
print(len(Larr))
print(len(tarr))
print(len(Earr))
print(max(Larr))
print(max(Earr))

# create a new plot with default tools, using figure
p = figure(title="Angular Momentum computed two ways", plot_width=400, plot_height=400)
#p.line(tarr,1000*np.array(vxarr1), line_color="purple",legend="External")
#p.line(tarr,np.array(ycoord1),line_color="green")
#p.line(tarr,1000*np.array(vyarr1), line_color="blue")
#p.line(tarr,np.array(xcoord1), line_color="yellow")
#p.line(tarr,np.array(vxarr1)*np.array(ycoord1), line_color="orange")
#p.line(tarr,np.array(vyarr1)*np.array(xcoord1),line_color="black")
p.line(tarr,(np.array(vxarr1)*np.array(ycoord1)-np.array(vyarr1)*np.array(xcoord1)), line_color="cyan", legend="L external")
p.line(tarr,np.array(Larr),line_color="blue", legend="Internal")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

999
999
999
-4.330637660316054
0.0032938185187060425


In [18]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()


# create a new plot with default tools, using figure
p = figure(title="coordinates vs time", plot_width=400, plot_height=400)
p.line(tarr,np.array(xcoord1), line_color="purple",legend="x")
p.line(tarr,np.array(ycoord1), line_color="blue", legend="y")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

In [19]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
print(len(Larr))
print(len(tarr))
print(len(Earr))
print(max(Larr))
print(max(Earr))

# create a new plot with default tools, using figure
p = figure(title="velocity vs time", plot_width=400, plot_height=400)
p.line(tarr,np.array(vxarr1), line_color="purple",legend="x velocity")
p.line(tarr,np.array(vyarr1),line_color="blue", legend="y velocity")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

999
999
999
-4.330637660316054
0.0032938185187060425


In [20]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="Angular Momentum", plot_width=400, plot_height=400)
p.line(tarr,Larr,line_color="purple", legend="Angular Momentum")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

In [21]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
xmomentum=np.array(vxarr1)+np.array(vxarr2)
p = figure(title="X linear momentum", plot_width=400, plot_height=400)
p.triangle(tarr,xmomentum, size=1, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="X linear momentum")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results 

Loading BokehJS ...